In [2]:
import json
import requests
from bs4 import BeautifulSoup
import urllib.parse # Thư viện để xử lý link mã hóa

# Tên file của bạn
input_file = 'link_title.json'
output_file = 'link_title_content.json'

def get_wiki_content(url):
    try:
        # BƯỚC QUAN TRỌNG: Giải mã URL từ dạng %C6... sang tiếng Việt có dấu
        # Ví dụ: .../Ai_t%C6%B0_v%C3%A3n -> .../Ai_tư_vãn
        # Điều này giúp requests xử lý đường dẫn chính xác hơn
        decoded_url = urllib.parse.unquote(url)
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # Gọi request
        response = requests.get(decoded_url, headers=headers, timeout=15)
        
        if response.status_code != 200:
            return f"Error: HTTP {response.status_code}"
            
        # Sử dụng .content (byte) thay vì .text để tránh lỗi decode response
        soup = BeautifulSoup(response.content, 'html.parser')
        
        content_div = soup.find('div', {'id': 'mw-content-text'})
        if not content_div:
            return ""

        parser_output = content_div.find('div', class_='mw-parser-output')
        extracted_text = []
        
        if parser_output:
            for element in parser_output.children:
                # Dừng khi gặp phần Tham khảo
                if element.name == 'h2':
                    if element.find('span', {'id': 'Tham_khảo'}) or 'Tham khảo' in element.get_text():
                        break

                # Lấy nội dung
                if element.name in ['p', 'h2', 'h3', 'h4']:
                    text = element.get_text().strip()
                    if text:
                        extracted_text.append(text)
                        
        return "\n\n".join(extracted_text)

    except Exception as e:
        return f"Error Exception: {str(e)}"

# --- PHẦN CHẠY CHÍNH ---

try:
    # 1. Đọc file JSON với encoding='utf-8' (Bắt buộc để tránh lỗi decode tiếng Việt)
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"Đã đọc {len(data)} link. Bắt đầu xử lý...")

    # 2. Vòng lặp
    for index, item in enumerate(data):
        print(f"[{index+1}/{len(data)}] Đang tải: {item.get('text', 'No Title')}")
        
        # Lấy link và xử lý
        link = item.get('href', '')
        if link:
            item['content'] = get_wiki_content(link)
        else:
            item['content'] = ""

    # 3. Lưu file kết quả cũng với encoding='utf-8'
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
    print(f"Xong! Kiểm tra file {output_file}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file {input_file}. Hãy chắc chắn file json nằm cùng thư mục code.")
except json.JSONDecodeError:
    print(f"Lỗi: File {input_file} không đúng định dạng JSON.")
except Exception as e:
    print(f"Lỗi không xác định: {str(e)}")

Đã đọc 59 link. Bắt đầu xử lý...
[1/59] Đang tải: Ai tư vãn
[2/59] Đang tải: Bác ơi!
[3/59] Đang tải: Bài thơ về tiểu đội xe không kính
[4/59] Đang tải: Bình Ngô đại cáo
[5/59] Đang tải: Bông hồng cài áo
[6/59] Đang tải: Cái mặt không chơi được
[7/59] Đang tải: Câu chuyện đối đáp của người tiều phu ở núi Na
[8/59] Đang tải: Châu bản triều Nguyễn
[9/59] Đang tải: Chí Phèo
[10/59] Đang tải: Chiếc thuyền ngoài xa
[11/59] Đang tải: Chiếu thư đánh Chiêm
[12/59] Đang tải: Chinh phụ ngâm
[13/59] Đang tải: Chuyện người con gái Nam Xương
[14/59] Đang tải: Công dư tiệp ký
[15/59] Đang tải: Dế Mèn phiêu lưu ký
[16/59] Đang tải: Đất nước đứng lên
[17/59] Đang tải: Đất rừng phương Nam
[18/59] Đang tải: Gia Định phú
[19/59] Đang tải: Gia Định thất thủ vịnh
[20/59] Đang tải: Hòn Đất (tiểu thuyết)
[21/59] Đang tải: Hồn Trương Ba, da hàng thịt
[22/59] Đang tải: Hương rừng Cà Mau
[23/59] Đang tải: Kim Thạch kỳ duyên
[24/59] Đang tải: Kính vạn hoa (truyện)
[25/59] Đang tải: Làm đĩ (tiểu thuyết)
[26/59] Đ

In [4]:
import re
def clean_text(text):
    if not isinstance(text, str) or not text:
        return ""
    
    # 1. Xóa các chú thích dạng [1], [12], [103]
    # Regex: \[ tìm dấu [, \d+ tìm số, \] tìm dấu ]
    text = re.sub(r'\[\d+\]', '', text)
    
    # 2. Xóa dấu ngoặc kép " (Thay bằng rỗng)
    text = text.replace('"', '')
    
    # 3. Xóa ký tự xuống dòng (Thay bằng khoảng trắng để nối chuỗi)
    text = text.replace('\n', ' ').replace('\r', ' ')
    
    # 4. Xóa ký tự KHÔNG PHẢI Latinh/Tiếng Việt
    # Logic: Tìm những ký tự KHÔNG nằm trong danh sách cho phép và thay bằng rỗng.
    # Danh sách giữ lại: 
    # - \w: Chữ cái và số (a-z, A-Z, 0-9)
    # - \s: Khoảng trắng
    # - \u00C0-\u1EF9: Dải Unicode chứa các ký tự Tiếng Việt có dấu
    # - .,!?:;()-_: Các dấu câu cơ bản
    text = re.sub(r'[^\w\s\u00C0-\u1EF9.,!?;:()\-]', '', text)
    
    # BƯỚC PHỤ: Chuẩn hóa khoảng trắng
    # (Xóa khoảng trắng thừa do việc xóa ký tự để lại, ví dụ "  " thành " ")
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Test thử hàm với chuỗi rác
test_str = 'Đây là ví dụ [1] có tiếng Nhật 日本 và dấu "ngoặc kép".\nXuống dòng.'
print("Gốc:", test_str)
print("Sạch:", clean_text(test_str))

Gốc: Đây là ví dụ [1] có tiếng Nhật 日本 và dấu "ngoặc kép".
Xuống dòng.
Sạch: Đây là ví dụ có tiếng Nhật 日本 và dấu ngoặc kép. Xuống dòng.


In [5]:
# Tên file đầu vào và đầu ra
input_file = 'link_title_content.json' # Thay tên file của bạn nếu khác
output_file = 'link_title_clean.json'

try:
    # Đọc file
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"Đang xử lý {len(data)} dòng dữ liệu...")
    
    # Duyệt và làm sạch từng phần tử
    count_processed = 0
    for item in data:
        original_content = item.get('content', '')
        
        # Gọi hàm clean_text đã định nghĩa ở trên
        cleaned_content = clean_text(original_content)
        
        # Cập nhật lại vào json
        item['content'] = cleaned_content
        count_processed += 1

    print("✅ Đã tiền xử lý xong!")
    
    # Xem thử 1 mẫu kết quả đầu tiên
    if len(data) > 0:
        print("\n--- Mẫu kết quả (Item 0) ---")
        print(data[0]['content'][:200] + "...") # In 200 ký tự đầu

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file '{input_file}'")
except Exception as e:
    print(f"❌ Lỗi: {str(e)}")

Đang xử lý 59 dòng dữ liệu...
✅ Đã tiền xử lý xong!

--- Mẫu kết quả (Item 0) ---
Ai tư vãn (chữ Hán: 哀思挽) là một tác phẩm trong văn chương Việt Nam, viết bằng chữ Nôm. Người ta tương truyền bài thơ này là do Bắc Cung Hoàng hậu Lê Ngọc Hân viết khóc phu quân là Quang Trung Hoàng đế...


In [6]:
try:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ Đã lưu file sạch vào: {output_file}")
except Exception as e:
    print(f"❌ Lỗi khi lưu file: {str(e)}")

✅ Đã lưu file sạch vào: link_title_clean.json


# tác giả

In [1]:
pip install requests beautifulsoup4


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time

# ==============================================================================
# 1. INPUT HTML
# ==============================================================================
html_input = """
<div class="div-col" style="column-width: 16em;">
<ul><li><a href="/wiki/Th%C3%A2m_T%C3%A2m" title="Thâm Tâm">Thâm Tâm</a></li>
<li><a href="/wiki/Ng%C3%B4_T%E1%BA%A5t_T%E1%BB%91" title="Ngô Tất Tố">Ngô Tất Tố</a></li>
<li><a href="/wiki/Nam_Tr%C3%A2n" title="Nam Trân">Nam Trân</a></li>
<li><a href="/wiki/Nguy%E1%BB%85n_B%C3%ADnh" title="Nguyễn Bính">Nguyễn Bính</a></li>
<li><a href="/wiki/D%C6%B0%C6%A1ng_H%C6%B0%E1%BB%9Bng" title="Dương Hướng">Dương Hướng</a></li>
<li><a href="/wiki/Thu_B%E1%BB%93n_(nh%C3%A0_th%C6%A1)" title="Thu Bồn (nhà thơ)">Thu Bồn</a></li>
<li><a href="/wiki/Nguy%E1%BB%85n_V%C4%83n_B%E1%BB%95ng_(nh%C3%A0_v%C4%83n)" title="Nguyễn Văn Bổng (nhà văn)">Nguyễn Văn Bổng</a></li>
<li><a href="/wiki/V%C4%83n_Cao" title="Văn Cao">Văn Cao</a></li>
<li><a href="/wiki/V%C5%A9_%C4%90%C3%ACnh_Li%C3%AAn" title="Vũ Đình Liên">Vũ Đình Liên</a></li>
<li><a href="/wiki/Ho%C3%A0ng_C%E1%BA%A7m_(nh%C3%A0_th%C6%A1)" title="Hoàng Cầm (nhà thơ)">Hoàng Cầm</a></li>
<li><a href="/wiki/Ho%C3%A0ng_Nhu%E1%BA%ADn_C%E1%BA%A7m" title="Hoàng Nhuận Cầm">Hoàng Nhuận Cầm</a></li>
<li><a href="/wiki/Huy_C%E1%BA%ADn" title="Huy Cận">Huy Cận</a></li>
<li><a href="/wiki/Minh_Chuy%C3%AAn_(nh%C3%A0_v%C4%83n)" title="Minh Chuyên (nhà văn)">Minh Chuyên</a></li>
<li><a href="/wiki/Xu%C3%A2n_Di%E1%BB%87u" title="Xuân Diệu">Xuân Diệu</a></li>
<li><a href="/wiki/Ph%E1%BA%A1m_Ti%E1%BA%BFn_Du%E1%BA%ADt" title="Phạm Tiến Duật">Phạm Tiến Duật</a></li>
<li><a href="/wiki/Ph%E1%BA%A1m_Hoa" title="Phạm Hoa">Phạm Hoa</a></li>
<li><a href="/wiki/Nguy%E1%BB%85n_%C4%90%C4%83c_M%E1%BA%ADu" title="Nguyễn Đức Mậu">Nguyễn Đức Mậu</a></li>
<li><a href="/wiki/Quang_D%C5%A9ng" class="mw-disambig" title="Quang Dũng">Quang Dũng</a></li>
<li><a href="/wiki/Nguy%E1%BB%85n_Khoa_%C4%90i%E1%BB%81m" title="Nguyễn Khoa Điềm">Nguyễn Khoa Điềm</a></li>
<li><a href="/wiki/%C4%90o%C3%A0n_Gi%E1%BB%8Fi" title="Đoàn Giỏi">Đoàn Giỏi</a></li>
<li><a href="/wiki/Thanh_H%E1%BA%A3i_(nh%C3%A0_th%C6%A1)" title="Thanh Hải (nhà thơ)">Thanh Hải</a></li>
<li><a href="/wiki/T%E1%BA%BF_Hanh" title="Tế Hanh">Tế Hanh</a></li>
<li><a href="/wiki/T%C3%B4_Ho%C3%A0i" title="Tô Hoài">Tô Hoài</a></li>
<li><a href="/wiki/Nguy%E1%BB%85n_C%C3%B4ng_Hoan" title="Nguyễn Công Hoan">Nguyễn Công Hoan</a></li>
<li><a href="/wiki/Nguy%C3%AAn_H%E1%BB%93ng" title="Nguyên Hồng">Nguyên Hồng</a></li>
<li><a href="/wiki/Ch%C3%ADnh_H%E1%BB%AFu" title="Chính Hữu">Chính Hữu</a></li>
<li><a href="/wiki/T%E1%BB%91_H%E1%BB%AFu" title="Tố Hữu">Tố Hữu</a></li>
<li><a href="/wiki/Nguy%E1%BB%85n_Kh%E1%BA%A3i_(nh%C3%A0_v%C4%83n)" class="mw-redirect" title="Nguyễn Khải (nhà văn)">Nguyễn Khải</a></li>
<li><a href="/wiki/Ma_V%C4%83n_Kh%C3%A1ng" title="Ma Văn Kháng">Ma Văn Kháng</a></li>
<li><a href="/wiki/Tr%E1%BA%A7n_%C4%90%C4%83ng_Khoa_(nh%C3%A0_th%C6%A1)" title="Trần Đăng Khoa (nhà thơ)">Trần Đăng Khoa</a></li>
<li><a href="/wiki/V%C5%A9_Khi%C3%AAu" title="Vũ Khiêu">Vũ Khiêu</a></li>
<li><a href="/wiki/Nguy%E1%BB%85n_Ng%E1%BB%8Dc_K%C3%BD" title="Nguyễn Ngọc Ký">Nguyễn Ngọc Ký</a></li>
<li><a href="/wiki/Chu_Lai_(nh%C3%A0_v%C4%83n)" title="Chu Lai (nhà văn)">Chu Lai</a></li>
<li><a href="/wiki/Kim_L%C3%A2n" title="Kim Lân">Kim Lân</a></li>
<li><a href="/wiki/Tr%E1%BA%A7n_Huy_Li%E1%BB%87u" title="Trần Huy Liệu">Trần Huy Liệu</a></li>
<li><a href="/wiki/Th%E1%BA%BF_L%E1%BB%AF" title="Thế Lữ">Thế Lữ</a></li>
<li><a href="/wiki/%C4%90%E1%BA%B7ng_Thai_Mai" title="Đặng Thai Mai">Đặng Thai Mai</a></li>
<li><a href="/wiki/Nguy%E1%BB%85n_%C4%90%C4%83ng_M%E1%BA%A1nh" title="Nguyễn Đăng Mạnh">Nguyễn Đăng Mạnh</a></li>
<li><a href="/wiki/T%C3%BA_M%E1%BB%A1" title="Tú Mỡ">Tú Mỡ</a></li>
<li><a href="/wiki/Th%C3%A9p_M%E1%BB%9Bi" title="Thép Mới">Thép Mới</a></li>
<li><a href="/wiki/V%C5%A9_T%C3%BA_Nam" title="Vũ Tú Nam">Vũ Tú Nam</a></li>
<li><a href="/wiki/Nguy%E1%BB%85n_Th%E1%BB%8B_H%E1%BB%93ng_Ng%C3%A1t" title="Nguyễn Thị Hồng Ngát">Nguyễn Thị Hồng Ngát</a></li>
<li><a href="/wiki/Nguy%C3%AAn_Ng%E1%BB%8Dc" title="Nguyên Ngọc">Nguyên Ngọc</a></li>
<li><a href="/wiki/B%E1%BA%A3o_Ninh" title="Bảo Ninh">Bảo Ninh</a></li>
<li><a href="/wiki/Nguy%E1%BB%85n_Tr%E1%BB%8Dng_O%C3%A1nh" title="Nguyễn Trọng Oánh">Nguyễn Trọng Oánh</a></li>
<li><a href="/wiki/V%C5%A9_Ng%E1%BB%8Dc_Phan" title="Vũ Ngọc Phan">Vũ Ngọc Phan</a></li>
<li><a href="/wiki/Ho%C3%A0ng_Ng%E1%BB%8Dc_Ph%C3%A1ch" title="Hoàng Ngọc Phách">Hoàng Ngọc Phách</a></li>
<li><a href="/wiki/Ho%C3%A0ng_Ph%E1%BB%A7_Ng%E1%BB%8Dc_T%C6%B0%E1%BB%9Dng" title="Hoàng Phủ Ngọc Tường">Hoàng Phủ Ngọc Tường</a></li>
<li><a href="/wiki/Thanh_Th%E1%BA%A3o_(nh%C3%A0_th%C6%A1)" title="Thanh Thảo (nhà thơ)">Thanh Thảo</a></li>
<li><a href="/wiki/Xu%C3%A2n_S%C3%A1ch" title="Xuân Sách">Xuân Sách</a></li>
<li><a href="/wiki/H%C3%A0_Minh_Tu%C3%A2n" title="Hà Minh Tuân">Hà Minh Tuân</a></li></ul>
</div>
"""

# ==============================================================================
# 2. CẤU HÌNH & HÀM XỬ LÝ
# ==============================================================================
BASE_URL = "https://vi.wikipedia.org"
OUTPUT_FILE = "dataset_hoi_nha_van.json"

# GIẢ LẬP TRÌNH DUYỆT (QUAN TRỌNG)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_links_from_html(html_str):
    soup = BeautifulSoup(html_str, 'html.parser')
    links = []
    a_tags = soup.select('li a')
    for a in a_tags:
        href = a.get('href')
        title = a.get('title')
        if href and href.startswith('/wiki/') and 'redlink' not in href:
            full_link = BASE_URL + href
            links.append({"name": title, "url": full_link})
    
    # Lọc trùng lặp
    unique_links = {v['url']: v for v in links}.values()
    return list(unique_links)

def scrape_and_split_content(author_list):
    final_dataset = []
    global_id = 1
    total = len(author_list)
    
    print(f"-> Tìm thấy {total} tác giả.")
    print("-" * 50)
    
    for i, author in enumerate(author_list):
        url = author['url']
        name = author['name']
        
        print(f"[{i+1}/{total}] Đang tải: {name} ... ", end="")
        
        try:
            # THÊM HEADERS VÀO YÊU CẦU
            response = requests.get(url, headers=HEADERS, timeout=10)
            
            if response.status_code != 200:
                print(f"Lỗi HTTP {response.status_code}")
                continue

            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Lấy tất cả thẻ p trong nội dung chính
            # Wikipedia thường dùng div mw-parser-output bên trong mw-content-text
            content_div = soup.find('div', {'id': 'mw-content-text'})
            if not content_div:
                print("Không tìm thấy nội dung.")
                continue

            paragraphs = content_div.find_all('p')
            
            count_local = 0
            for p in paragraphs:
                text = p.get_text()
                # Cleaning
                text = re.sub(r'\[\d+\]', '', text)
                text = re.sub(r'\s+', ' ', text).strip()
                
                if not text: continue
                
                # Splitting
                sentences = re.split(r'(?<=[.!?])\s+', text)
                
                for sent in sentences:
                    # Giảm điều kiện lọc xuống 15 ký tự để bắt được nhiều hơn
                    if len(sent) > 15:
                        final_dataset.append({
                            "id": global_id,
                            "source": name,
                            "text": sent
                        })
                        global_id += 1
                        count_local += 1
            
            print(f"-> Lấy được {count_local} câu.")
            time.sleep(0.2) # Nghỉ nhẹ
            
        except Exception as e:
            print(f"Lỗi: {e}")
            continue
            
    print("-" * 50)
    return final_dataset

# ==============================================================================
# 3. MAIN
# ==============================================================================
if __name__ == "__main__":
    authors = get_links_from_html(html_input)
    
    if authors:
        dataset = scrape_and_split_content(authors)
        
        if len(dataset) > 0:
            with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
                json.dump(dataset, f, ensure_ascii=False, indent=2)
            print(f"✅ THÀNH CÔNG! Đã lưu {len(dataset)} câu vào '{OUTPUT_FILE}'.")
        else:
            print("⚠️ Vẫn không lấy được câu nào. Hãy kiểm tra lại kết nối mạng.")
    else:
        print("Không parse được HTML đầu vào.")

-> Tìm thấy 51 tác giả.
--------------------------------------------------
[1/51] Đang tải: Thâm Tâm ... -> Lấy được 27 câu.
[2/51] Đang tải: Ngô Tất Tố ... -> Lấy được 68 câu.
[3/51] Đang tải: Nam Trân ... -> Lấy được 16 câu.
[4/51] Đang tải: Nguyễn Bính ... -> Lấy được 138 câu.
[5/51] Đang tải: Dương Hướng ... -> Lấy được 13 câu.
[6/51] Đang tải: Thu Bồn (nhà thơ) ... -> Lấy được 32 câu.
[7/51] Đang tải: Nguyễn Văn Bổng (nhà văn) ... -> Lấy được 24 câu.
[8/51] Đang tải: Văn Cao ... -> Lấy được 328 câu.
[9/51] Đang tải: Vũ Đình Liên ... -> Lấy được 23 câu.
[10/51] Đang tải: Hoàng Cầm (nhà thơ) ... -> Lấy được 28 câu.
[11/51] Đang tải: Hoàng Nhuận Cầm ... -> Lấy được 12 câu.
[12/51] Đang tải: Huy Cận ... -> Lấy được 35 câu.
[13/51] Đang tải: Minh Chuyên (nhà văn) ... -> Lấy được 40 câu.
[14/51] Đang tải: Xuân Diệu ... -> Lấy được 106 câu.
[15/51] Đang tải: Phạm Tiến Duật ... -> Lấy được 26 câu.
[16/51] Đang tải: Phạm Hoa ... -> Lấy được 18 câu.
[17/51] Đang tải: Nguyễn Đức Mậu ... Lỗi 

In [7]:
import json
import os

input_file = 'dataset_hoi_nha_van.json'
output_file = 'dataset_hoi_nha_van_final.json'

def limit_sentences_per_author(input_path, output_path, max_sentences=20):
    try:
        # 1. Đọc dữ liệu
        if not os.path.exists(input_path):
            print(f"Lỗi: Không tìm thấy file {input_path}")
            return

        with open(input_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        print(f"Tổng số câu ban đầu: {len(data)}")
        
        # 2. Lọc dữ liệu
        filtered_data = []
        author_counts = {} # Biến đếm số câu của từng tác giả
        new_id = 1
        
        for item in data:
            author = item.get('source', 'Unknown')
            
            # Khởi tạo đếm nếu gặp tác giả mới
            if author not in author_counts:
                author_counts[author] = 0
            
            # Kiểm tra điều kiện: Chưa đủ 30 câu thì lấy
            if author_counts[author] < max_sentences:
                # Tạo bản ghi mới để đánh lại ID luôn cho đẹp
                new_item = {
                    "id": new_id,
                    "source": author,
                    "text": item['text']
                }
                filtered_data.append(new_item)
                
                # Tăng biến đếm và ID
                author_counts[author] += 1
                new_id += 1
        
        # 3. Lưu file mới
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(filtered_data, f, ensure_ascii=False, indent=2)
            
        print("-" * 30)
        print(f"Đã xử lý xong!")
        print(f"File gốc: {len(data)} câu")
        print(f"File mới: {len(filtered_data)} câu (đã lưu tại '{output_path}')")
        print("-" * 30)
        
        # In thống kê chi tiết vài tác giả để kiểm tra
        print("Thống kê số lượng câu per tác giả (Top 5):")
        for auth, count in list(author_counts.items())[:5]:
            print(f"- {auth}: {count} câu")

    except Exception as e:
        print(f"Có lỗi xảy ra: {e}")

# --- CHẠY ---
limit_sentences_per_author(input_file, output_file)

Tổng số câu ban đầu: 2263
------------------------------
Đã xử lý xong!
File gốc: 2263 câu
File mới: 913 câu (đã lưu tại 'dataset_hoi_nha_van_final.json')
------------------------------
Thống kê số lượng câu per tác giả (Top 5):
- Thâm Tâm: 20 câu
- Ngô Tất Tố: 20 câu
- Nam Trân: 16 câu
- Nguyễn Bính: 20 câu
- Dương Hướng: 13 câu
